In [1]:
!pip install langchain pinecone-client google-generativeai tqdm openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 1.4 MB/s eta 0:00:00


In [5]:
%pip install --upgrade --quiet  langchain-pinecone langchain-openai pinecone-notebooks langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 1.7 MB/s eta 0:00:00


In [8]:
from google.colab import userdata
import os

from pinecone import Pinecone, ServerlessSpec

pinecone_api_key = userdata.get("PINECONE_API")

pc = Pinecone(api_key=pinecone_api_key)

In [9]:
import time

index_name = "first-rag-project"  # change if desired

pc.create_index(
        name=index_name,
        dimension=768, #as we are using gemini it has 768 dimensions
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
index = pc.Index(index_name)

#Google embeddings

In [10]:
%pip install --upgrade --quiet  langchain-google-genai

In [11]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]

[0.05168594419956207,
 -0.030764883384108543,
 -0.03062233328819275,
 -0.02802734263241291,
 0.01813093200325966]

#Storing database

In [12]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [15]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

['2599b85c-f1ad-4c2a-82c4-3db043faf82c',
 'c0c15f10-f1a6-4ffe-b0d3-95b595c2fc34',
 'd11563b7-7bf9-4969-a61d-d260c5ec4772',
 '3a43cbfd-df4b-49a5-983f-379b024ebeb3',
 'dab1df90-c11a-4cb8-9739-206fd521f5d9',
 'd139e759-b360-43d7-b22e-036b07c43dde',
 '48249c0e-1332-4254-8767-9f28a9df259d',
 'b8236ae7-9ae5-477f-aeb8-675a7f342c1a',
 '78a3f372-3a5c-4309-86db-a5160161434d',
 '07f1cdf4-e00c-4dde-8be3-42babc4bf268']

#Basic RAG done

In [18]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]
* I have a bad feeling I am going to get deleted :( [{'source': 'tweet'}]
* I had chocalate chip pancakes and scrambled eggs for breakfast this morning. [{'source': 'tweet'}]


In [16]:
results = vector_store.similarity_search(
    "LangChain provides abstractions to make working with LLMs easy",
    k=2,
    filter={"source": "tweet"},
)
for res in results:
    print(f"* {res.page_content} [{res.metadata}]")

* LangGraph is the best framework for building stateful, agentic applications! [{'source': 'tweet'}]
* Building an exciting new project with LangChain - come check it out! [{'source': 'tweet'}]


In [17]:
results = vector_store.similarity_search_with_score(
    "Will it be hot tomorrow?", k=1, filter={"source": "news"}
)
for res, score in results:
    print(f"* [SIM={score:3f}] {res.page_content} [{res.metadata}]")

* [SIM=0.667716] The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees. [{'source': 'news'}]


#Passing to the Model for the best answer

In [44]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [45]:
def answer_to_user(query:str):
  # Vector search
  vector_results = vector_store.similarity_search(query,k=2)
  # Passing to model
  answer = llm.invoke(f"Answer to the following question {query}, here some reference results {results}")
  return answer

In [50]:
question = "Will it be hot tomorrow?"
answer = answer_to_user(question)
answer

AIMessage(content='Based on the provided text, no, it will not be hot tomorrow.  The forecast predicts a high of 62 degrees, which is not considered hot.  The second document is irrelevant to the weather.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-d39185eb-dd8d-4f92-97d1-bb2d28b0f033-0', usage_metadata={'input_tokens': 144, 'output_tokens': 44, 'total_tokens': 188, 'input_token_details': {'cache_read': 0}})

In [49]:
answer.content

'Based on the provided text, no, it will not be hot tomorrow.  The forecast predicts a high of 62 degrees, which is not considered hot.  The second document is irrelevant to the weather.'